In [318]:
import json

import numpy as np
import torch
import matplotlib.pyplot as plt

from scipy import stats

# Increase Confidence

In [319]:
# === Softmax ===
n_classes = 10
random_expert_idx = 0
probs_softmax = []
confs = []
exps = []
true = []
path = "softmax_increase_confidence/"
n_experts = 4
p_experts = [0.2, 0.4, 0.6, 0.8, 0.95]
for p in p_experts:
    model_name = '_' + str(p) + '_confidence'
    with open(path + 'confidence_multiple_experts' + model_name + '.txt', 'r') as f:
        conf = json.loads(json.load(f))
    with open(path + 'expert_predictions_multiple_experts' + model_name + '.txt', 'r') as f:
        exp_pred = json.loads(json.load(f))
    with open(path + 'true_label_multiple_experts' + model_name + '.txt', 'r') as f:
        true_label = json.loads(json.load(f))
    true.append(true_label['test'])
    exps.append(exp_pred['test'])
    c = torch.tensor(conf['test'])
    print(c.shape)
    # DANI Correction ===
    c = c.softmax(dim=1)
    probs_softmax.append(c)
    # DANI Correction ===

    temp = 0
    for i in range(n_experts):
        temp += c[:, (n_classes + n_experts) - (i + 1)]
    prob = c / (1.0 - temp).unsqueeze(-1)
    confs.append(prob)


torch.Size([9216, 14])
torch.Size([9216, 14])
torch.Size([9216, 14])
torch.Size([9216, 14])
torch.Size([9216, 14])


### Use c, not conf, because we do not apply temperature scaling

In [320]:
# 1 expert rando, 3 with prob 0.8 correct
probs = probs_softmax[-1]
experts = exps[-1]
# experts = experts[::-1]  # reverse order!
y_true = true[-1]

In [ ]:
probs[:5,10:]

In [ ]:
np.array(experts)

In [ ]:
y_true[:5]

In [ ]:
n_val = int(0.8 * len(y_true))
n_test = len(y_true) - n_val
print("N val:{}".format(n_val))
print("N test:{}".format(n_test))

In [ ]:
probs

# Validation

In [ ]:
## 2. get Q_hat

n_classes_exp = n_classes + n_experts
probs_val = probs[:n_val, 10:]

# experts_val = experts[::-1]  # IMPORTANT! swap to match prob ordering
experts_val = experts
experts_val = [exp[:n_val] for exp in experts_val]

y_true_val = y_true[:n_val]

# === Only on deferred samples
_, predicted = torch.max(probs[:n_val].data, 1)
r = (predicted >= n_classes_exp - n_experts)

# Filter 
probs_val = probs_val[r]
experts_val = [np.array(exp)[r] for exp in experts_val]
y_true_val = np.array(y_true_val)[r]

# Model expert probs ===
# Sort J model outputs for experts
probs_experts = probs[:n_val, 10:]
probs_experts = probs_experts[r]
c, pi = probs_experts.sort(dim=1, descending=True)

# Correctness experts ===
# Check if experts are correct 
correct_exp = (np.array(experts_val) == np.array(y_true_val)).T
# idx for correct experts: [[0,1,2], [1,2], [], ...]
correct_exp_idx = [np.where(correct_exp_i)[0] for correct_exp_i in correct_exp]

# obtain the last expert to be retrieved. If empty, then add all values.
# indexes are not the real expert index, but the sorted indexes, e.g. [[1, 0 ,2],  [1,0], [], ...]
pi_corr_exp = [probs_experts[i, corr_exp].sort(descending=True)[1] for i, corr_exp in enumerate(correct_exp)]
pi_corr_exp_stop = [pi_corr_exp_i[-1] if len(pi_corr_exp_i)!=0 else -1 for pi_corr_exp_i in pi_corr_exp]  # last expert

# obtain real expert index back, e.g. [2,1,-1,...]
pi_stop = [correct_exp_idx[i][pi_corr_exp_stop_i] if len(correct_exp_idx[i])!=0 else -1 for i, pi_corr_exp_stop_i in enumerate(pi_corr_exp_stop)]


# =========
n_val = n_val
alpha = 0.1
scores = sort.cumsum(dim=1).gather(1, pi.argsort(1))[range(len(torch.tensor(pi_stop))), torch.tensor(pi_stop)]
qhat = torch.quantile(scores, np.ceil((r.sum() + 1) * (1 - alpha)) / r.sum(), interpolation="higher")

qhat

In [ ]:
probs_val

In [ ]:
probs[:,10:]

In [ ]:
sort.cumsum(dim=1).gather(1, pi.argsort(1))

In [ ]:
y_true_val

In [ ]:
np.array(experts_val)

# Test

In [ ]:
n_classes = 10
n_experts = 4
n_classes_exp = n_classes + n_experts

probs_test = probs[n_val:, n_classes:]
experts_test = [exp[n_val:] for exp in experts]
y_true_test = y_true[n_val:]


In [ ]:
probs_test.sum(1)

In [ ]:
# === Only on deferred samples
_, predicted = torch.max(probs[n_val:].data, 1)
r_test = (predicted >= n_classes_exp - n_experts)

# Filter 
probs_test = probs_test[r_test]
experts_test = [np.array(exp)[r_test] for exp in experts_test]
y_true_test = np.array(y_true_test)[r_test]

# Sort J model outputs for experts. sorted probs and sorted indexes
sort_test, pi_test = probs_test.sort(dim=1, descending=True)
# Get last sorted index to be below Q_hat
pi_stop = (sort_test.cumsum(dim=1) <= qhat).sum(axis=1)

# Prediction sets
prediction_sets = [(pi_test[i][:(pi_stop[i])]).numpy() for i in range(pi_stop.shape[0])]  # not allow empty sets
prediction_sets[:5]

In [ ]:
r.sum()

In [ ]:
a = np.array([len(prediction_sets_i) for prediction_sets_i in prediction_sets])
import seaborn as sns

sns.histplot(a)

In [ ]:
np.array(experts_test)[:,:5]

# Accuracy

### Accuracy w/o Conformal on deferred samples

In [ ]:
correct = 0
correct_sys = 0
exp = 0
exp_total = 0
total = 0
real_total = 0
alone_correct = 0
#  === Individual Expert Accuracies === #
expert_correct_dic = {k: 0 for k in range(len(experts_test))}
expert_total_dic = {k: 0 for k in range(len(experts_test))}

In [ ]:
# 1 expert rando, 3 with prob 0.9 correct
probs = probs_softmax[-1]
experts = exps[-1]
# experts = experts[::-1] 
y_true = true[-1]

In [ ]:
n_classes = 10
n_experts = 4
n_classes_exp = n_classes + n_experts

probs_test = probs[n_val:]
experts_test = [exp[n_val:] for exp in experts]
y_true_test = y_true[n_val:]


In [ ]:
probs.shape

In [ ]:
# Predicted value 
_, predicted = torch.max(probs_test.data, 1)
# Classifier alone prediction
_, prediction = torch.max(probs_test.data[:, :(n_classes_exp - n_experts)],1)

### w Conformal Prediction

In [ ]:
probs.sum(1)

In [ ]:
labels = y_true_test

# Predicted value 
_, predicted = torch.max(probs_test.data, 1)
# Classifier alone prediction
_, prediction = torch.max(probs_test.data[:, :(n_classes_exp - n_experts)],1)
for i in range(0, n_test):
    r = (predicted[i].item() >= n_classes_exp - len(experts_test))
    alone_correct += (prediction[i] == labels[i]).item()
    if r == 0:
        total += 1
        correct += (predicted[i] == labels[i]).item()
        correct_sys += (predicted[i] == labels[i]).item()
        
    if r == 1:
        # Conformal prediction ===        
        # Sort J model outputs for experts. sorted probs and sorted indexes
        sort_i, pi_i = probs_test[i,n_classes:].sort(descending=True)
        # Get last sorted index to be below Q_hat
        pi_stop_i = (sort_i.cumsum(dim=0) <= qhat).sum()

        # Prediction sets
        prediction_set_i = (pi_i[:(pi_stop_i)]).numpy()  # not allow empty sets        
#         print(len(prediction_set_i))
        ensemble_expert_pred_i = np.array(experts_test)[prediction_set_i][:, i]
        exp_prediction = stats.mode(ensemble_expert_pred_i).mode
        # Conformal prediction ===
        
        # Deferral accuracy: No matter expert ===
        exp += (exp_prediction == labels[i])
        exp_total += 1
        # Individual Expert Accuracy ===
        # expert_correct_dic[deferred_exp] += (exp_prediction == labels[i].item())
        # expert_total_dic[deferred_exp] += 1
        #
        correct_sys += (exp_prediction == labels[i])
    real_total += 1
    
#  ===  Coverage  === #    
cov = str(total) + str(" out of") + str(real_total)

#  === Individual Expert Accuracies === #
expert_accuracies = {"expert_{}".format(str(k)): 100 * expert_correct_dic[k] / (expert_total_dic[k] + 0.0002) for k
             in range(len(experts_test))}






In [ ]:
exp_prediction

In [ ]:
sort_i

In [ ]:
print("Cov: {}".format(100 * (total/real_total)))

In [ ]:
expert_accuracies

In [ ]:
100 * exp / (exp_total + 0.0002),

In [ ]:
exp_total

In [ ]:
100 * correct_sys / real_total

### w/o Conformal Prediction

In [ ]:
correct = 0
correct_sys = 0
exp = 0
exp_total = 0
total = 0
real_total = 0
alone_correct = 0
#  === Individual Expert Accuracies === #
expert_correct_dic = {k: 0 for k in range(len(experts_test))}
expert_total_dic = {k: 0 for k in range(len(experts_test))}

In [ ]:
# 1 expert rando, 3 with prob 0.9 correct
probs = probs_softmax[-1]
experts = exps[-1]
# experts = experts[::-1] 
y_true = true[-1]

In [ ]:
n_classes = 10
n_experts = 4
n_classes_exp = n_classes + n_experts

probs_test = probs[n_val:]
experts_test = [exp[n_val:] for exp in experts]
y_true_test = y_true[n_val:]


In [ ]:
# Predicted value 
_, predicted = torch.max(probs_test.data, 1)
# Classifier alone prediction
_, prediction = torch.max(probs_test.data[:, :(n_classes_exp - n_experts)],1)

In [ ]:
labels = y_true_test

# Predicted value 
_, predicted = torch.max(probs_test.data, 1)
# Classifier alone prediction
_, prediction = torch.max(probs_test.data[:, :(n_classes_exp - n_experts)],1)
for i in range(0, n_test):
    r = (predicted[i].item() >= n_classes_exp - len(experts_test))
    alone_correct += (prediction[i] == labels[i]).item()
    if r == 0:
        total += 1
        correct += (predicted[i] == labels[i]).item()
        correct_sys += (predicted[i] == labels[i]).item()
        
    if r == 1:
        deferred_exp = (predicted[i] - n_classes).item()  # reverse order, as in loss function
        # print("def exp: {}".format(deferred_exp))
        # deferred_exp = ((n_classes - 1) - predicted[i]).item()  # reverse order, as in loss function
        exp_prediction = experts_test[deferred_exp][i]
        # print("exp pred: {}".format(exp_prediction))
        #
        # Deferral accuracy: No matter expert ===
        exp += (exp_prediction == labels[i])
        # print("label: {}".format(labels[i]))

        exp_total += 1
        # Individual Expert Accuracy ===
        expert_correct_dic[deferred_exp] += (exp_prediction == labels[i])
        expert_total_dic[deferred_exp] += 1
        #
        correct_sys += (exp_prediction == labels[i])
    real_total += 1
    
#  ===  Coverage  === #    
cov = str(total) + str(" out of") + str(real_total)

#  === Individual Expert Accuracies === #
expert_accuracies = {"expert_{}".format(str(k)): 100 * expert_correct_dic[k] / (expert_total_dic[k] + 0.0002) for k
             in range(len(experts_test))}






In [ ]:
(predicted[i] - n_classes)

In [400]:
n_classes = 10
n_experts = 4
n_classes_exp = n_classes + n_experts

probs_test = probs[n_val:]
experts_test = [exp[n_val:] for exp in experts]
y_true_test = y_true[n_val:]


In [401]:
# Predicted value 
_, predicted = torch.max(probs_test.data, 1)
# Classifier alone prediction
_, prediction = torch.max(probs_test.data[:, :(n_classes_exp - n_experts)],1)

In [402]:
labels = y_true_test

# Predicted value 
_, predicted = torch.max(probs_test.data, 1)
# Classifier alone prediction
_, prediction = torch.max(probs_test.data[:, :(n_classes_exp - n_experts)],1)
for i in range(0, n_test):
    r = (predicted[i].item() >= n_classes_exp - len(experts_test))
    alone_correct += (prediction[i] == labels[i]).item()
    if r == 0:
        total += 1
        correct += (predicted[i] == labels[i]).item()
        correct_sys += (predicted[i] == labels[i]).item()
        
    if r == 1:
        deferred_exp = (predicted[i] - n_classes).item()  # reverse order, as in loss function
        # print("def exp: {}".format(deferred_exp))
        # deferred_exp = ((n_classes - 1) - predicted[i]).item()  # reverse order, as in loss function
        exp_prediction = experts_test[deferred_exp][i]
        # print("exp pred: {}".format(exp_prediction))
        #
        # Deferral accuracy: No matter expert ===
        exp += (exp_prediction == labels[i])
        # print("label: {}".format(labels[i]))

        exp_total += 1
        # Individual Expert Accuracy ===
        expert_correct_dic[deferred_exp] += (exp_prediction == labels[i])
        expert_total_dic[deferred_exp] += 1
        #
        correct_sys += (exp_prediction == labels[i])
    real_total += 1
    
#  ===  Coverage  === #    
cov = str(total) + str(" out of") + str(real_total)

#  === Individual Expert Accuracies === #
expert_accuracies = {"expert_{}".format(str(k)): 100 * expert_correct_dic[k] / (expert_total_dic[k] + 0.0002) for k
             in range(len(experts_test))}






In [403]:
(predicted[i] - n_classes)

tensor(-5)

In [404]:
expert_accuracies

{'expert_0': 0.0,
 'expert_1': 85.07449988880613,
 'expert_2': 82.43220964267664,
 'expert_3': 81.69931080640052}

In [405]:
100 * exp / (exp_total + 0.0002),

(82.45874588943153,)

In [406]:
exp

550

In [407]:
100 * correct_sys / real_total

88.01518438177874

### Accuracy w Conformal on deferred samples

In [28]:
qhat

tensor(2.4832)

In [29]:
probs_test[:,10:]

tensor([[0.9446, 0.9549, 0.9215, 0.1244],
        [0.9760, 0.9769, 0.9756, 0.1037],
        [0.9622, 0.9661, 0.9566, 0.1071],
        ...,
        [0.9315, 0.9366, 0.9419, 0.1079],
        [0.2346, 0.2542, 0.2841, 0.1010],
        [0.9251, 0.9364, 0.9127, 0.1279]])